## Load Libraries

In [46]:
import pandas as pd
import numpy as np
import random as rnd
import datetime

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
#sns.set_style('whitegrid')
%matplotlib inline

# plotly
import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import cufflinks as cf
cf.go_offline()

from fastai.tabular.all import *

# SKlearn
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm.sklearn import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import *
from sklearn.model_selection import GridSearchCV

from time import time


from tqdm import tqdm
from joblib import Parallel, delayed
from sklearn.base import clone

from tune_sklearn import TuneSearchCV
from tabulate import tabulate

## Load Data

In [47]:
filepath = "/Users/Kapil/PycharmProjects/stock-price"
train = pd.read_csv(filepath + "/data/train.csv")
test = pd.read_csv(filepath + "/data/test.csv")

In [48]:
SEED = 13
np.random.seed(SEED)

In [49]:
Y_train = train['Up']
#X_train = train.drop(['Up'], axis = 1)

## Functions
#### Preprocessing function

In [50]:
def LagPreproc(df):
    data = pd.DataFrame()
    data['Today'] = df['Close'].pct_change() * 100
    #data = data.rename("Today")
    for i in range(1,6):
        data['Lag'+str(i)] = data['Today'].shift(i)
    data = data.fillna(data.mean())
    return data

#### Specificity function

In [51]:
def specificity(y_truth, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_truth, y_pred).ravel()

    specificity = tn / (tn+fp)
    return specificity

#### Fit and Score function

In [52]:
def fit_and_score(model, train_index, valid_index, X, Y):
    """
    function to use in a joblib parallel env, takes the inputs and returns metrics
    return: (roc_train, roc_val, f1_train, f1_val)
    """
    # get X and y vectors
    x_train, x_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = Y.iloc[train_index], Y.iloc[valid_index]

    # over sample x
    #x_train, y_train = ros.fit_resample(x_train, y_train)

    # preprocess
#     x_train = ct.fit_transform(x_train)
#     x_valid = ct.transform(x_valid)

    # train model
    model.fit(x_train, y_train)

    # append scores
    y_train_preds = model.predict(x_train)
    y_valid_preds = model.predict(x_valid)

    f1_train = f1_score(y_train, y_train_preds)
    roc_train = roc_auc_score(y_train, model.predict_proba(x_train)[:,1])
    mcc_train = matthews_corrcoef(y_train, y_train_preds)
    acc_train = accuracy_score(y_train, y_train_preds)

    f1_val = f1_score(y_valid, y_valid_preds)
    roc_val = roc_auc_score(y_valid, model.predict_proba(x_valid)[:,1])
    mcc_val = matthews_corrcoef(y_valid, y_valid_preds)
    acc_val= accuracy_score(y_train, y_train_preds)

    log_line = 'Train score: ' + str(acc_train) + ' Validation score: ' + str(roc_val)
    tqdm.write(log_line)

    return roc_train, roc_val, f1_train, f1_val, mcc_train, mcc_val

#### Submission format function

In [ ]:
def submitformat(df, test_df):
    df[df > 0.5] = 1
    df[df <= 0.5] = 0
    df = pd.DataFrame(df)
    df.index = test_df.index
    df = df.iloc[5::6, :]
    df = df.astype(int)
    return df

### Preprocess train and test

In [53]:
X_train  = LagPreproc(train)
X_test = LagPreproc(test)

In [61]:
X_train.head()

,Today,Lag1,Lag2,Lag3,Lag4,Lag5
0,0.043343,0.043086,0.042972,0.043069,0.04265,0.042789
1,0.138013,0.043086,0.042972,0.043069,0.04265,0.042789
2,0.708801,0.138013,0.042972,0.043069,0.04265,0.042789
3,0.703812,0.708801,0.138013,0.043069,0.04265,0.042789
4,-0.446515,0.703812,0.708801,0.138013,0.04265,0.042789


## Models
#### Logistic Regression Tuning

In [54]:
penalty = ['l1', 'l2']
C = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
class_weight = [{1:0.5, 0:0.5}, {1:0.4, 0:0.6}, {1:0.6, 0:0.4}, {1:0.7, 0:0.3}]
solver = ['liblinear', 'saga']

param_grid = dict(penalty=penalty,
                  C=C,
                  class_weight=class_weight,
                  solver=solver)

grid = GridSearchCV(LogisticRegression(solver='liblinear'),
                    param_grid=param_grid,
                    scoring='roc_auc',
                    verbose=1,
                    n_jobs=-1)
grid_result = grid.fit(X_train, Y_train)

print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

model = LogisticRegression(grid_result.best_params_)

#### Decision Tree tuning

In [ ]:
config = {
    'min_samples_leaf': [15, 19, 23, 25, 28, 30, 35, 40],
    "max_depth": [5, 8, 10, 12]
}
model = TuneSearchCV(DecisionTreeClassifier(random_state=SEED),
                          config,
                          scoring='accuracy',
                          n_jobs=-1, cv =5,
                          search_optimization='bayesian',
                          verbose=1,
                          random_state=SEED)

#### Random Forest

In [11]:
config = {
    "n_estimators": [300, 400, 600, 650, 700, 750, 800, 1000],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5],
    "max_depth": [9, 12] ,
    #15, 18, 21, 24],
    "max_features": ['auto', 'sqrt']
}


model = TuneSearchCV(RandomForestClassifier(n_jobs= -1, verbose = 1,
                                                  random_state=SEED),
                          config,
                          scoring='accuracy',
                          n_jobs=-1, cv=5,
                          search_optimization='bayesian',
                          verbose=1,
                          random_state=SEED)


#### LightGBM

In [19]:
config = {
    "n_estimators": [400, 600, 700, 800, 1000],
    "colsample_bytree": [0.7, 0.8],
    "max_depth": [5, 10, 15, 20, 25],
    "num_leaves": [50, 100, 200],
    "reg_lambda": [1.1, 1.2, 1.3],
    "min_split_gain": [0.3, 0.4],
    "subsample": [0.7, 0.8, 0.9],
    "learning_rate": [0.05, 0.1]
}
model = TuneSearchCV(LGBMClassifier(n_jobs=-1, random_state=SEED),
                          config,
                          scoring='accuracy',
                          n_jobs=-1, cv=5,
                          search_optimization='bayesian',
                          verbose=1,
                          random_state=SEED)

#### XGBoost Tuning

In [13]:
config = {
    "n_estimators": [100, 200, 300, 400, 600, 700, 800],
    "colsample_bytree": [0.7, 0.8],
    "min_child_weight": [1,3,5,7] ,
    "max_depth": [3, 6, 9, 12],
    "learning_rate": [0.05, 0.1]
}

model = TuneSearchCV(XGBClassifier(n_jobs=-1, random_state=SEED),
                          config,
                          scoring='accuracy',
                          n_jobs=-1, cv=5,
                          search_optimization='bayesian',
                          verbose=1,
                          random_state=SEED)

#### SVC Tuning

In [32]:
config = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}

model = TuneSearchCV(SVC(n_jobs=-1, random_state=SEED),
                          config,
                          scoring='accuracy',
                          n_jobs=-1, cv = 5,
                          search_optimization='bayesian',
                          verbose=1,
                          random_state=SEED)

#### Knn Tuning

In [33]:
config = {'leaf_size' : list(range(1,100)),
          'n_neighbors' : list(range(1,100))
          'p':[1,2]
}

model = TuneSearchCV(KNeighborsClassifier(n_jobs=-1, random_state=SEED),
                          config,
                          scoring='accuracy',
                          n_jobs=-1, cv = 5,
                          search_optimization='bayesian',
                          verbose=1,
                          random_state=SEED)

## Parallel Process



kfolds = StratifiedKFold(n_splits=5, shuffle=True)
n_jobs = -1

st = time()

if n_jobs == 1:
    print('Running single process loop')
    res = []

    for train_index, valid_index in tqdm(kfolds.split(X_train, Y_train)):
        res.append(fit_and_score(clone(model), train_index, valid_index, X_train, Y_train))
else:
    print('Running parallel loop')
    parallel = Parallel(n_jobs=n_jobs, verbose=1)
    res = parallel(delayed(fit_and_score)(clone(model), train_idx, valid_idx, X_train, Y_train)
              for train_idx, valid_idx in kfolds.split(X_train, Y_train))


end = time()

print('Total time taken ', (end - st)/60, ' mins')

## Metrics

res = pd.DataFrame(res, columns=['ROC_train', 'ROC_val', 'F1_train', 'F1_val', 'MCC_train', 'MCC_val'])
res

print('Mean roc auc (train, valid) : ', res['ROC_train'].mean(), ', ', res['ROC_val'].mean())

## Training on Entire dataset

In [59]:
model.fit(X_train, Y_train)

DecisionTreeClassifier(max_depth=10, min_samples_leaf=30)

## Model Performance

In [60]:
# model_pipe.roc_auc_score(X_train,Y_train)
Y_train_preds = model.predict_proba(X_train)[:,1]
Y_train_preds = np.round(Y_train_preds).astype(int)
train_score = accuracy_score(Y_train,Y_train_preds)

Y_test_preds = model.predict_proba(X_test)[:,1]

print('Training score: ', train_score)
print(classification_report(Y_train, model.predict(X_train)))

Training score:  0.7124645781816231
              precision    recall  f1-score   support

           0       0.64      0.66      0.65       981
           1       0.67      0.65      0.66      1045

    accuracy                           0.65      2026
   macro avg       0.65      0.65      0.65      2026
weighted avg       0.65      0.65      0.65      2026

